# Example: Pipe Simulation

This notebook demonstrates a complete simulation of a simple water network. The network consists of a `Reservoir` connected to a `Pipe`, which in turn is connected to a `Gate` that acts as the outflow boundary. 

The simulation is orchestrated by the `SimulationHarness`, which manages the components, their connections (topology), and the simulation loop.

## Simulation Setup

1.  **Components**: We define three components:
    *   `Reservoir (res1)`: The water source, with an initial water level.
    *   `Pipe (pipe1)`: Connects the reservoir to the gate.
    *   `Gate (g1)`: The outflow point, with a fixed opening.
2.  **Topology**: We connect the components in a series: `res1 -> pipe1 -> g1`.
3.  **Harness**: The `SimulationHarness` is configured to run for a duration of 600 seconds with a 1-second time step.
4.  **Execution**: The harness first builds the simulation by topologically sorting the components and then runs the simulation step-by-step.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.simulation_identification.physical_objects.pipe import Pipe
from swp.core_engine.testing.simulation_harness import SimulationHarness

# 1. Define the components
reservoir = Reservoir(
    name="res1",
    initial_state={'volume': 25e6, 'water_level': 15.0},
    parameters={'surface_area': 1.5e6}
)

pipe = Pipe(
    name="pipe1",
    initial_state={'outflow': 0},
    parameters={
        'length': 1000,  # 1 km
        'diameter': 1.5, # m
        'friction_factor': 0.02
    }
)

gate = Gate(
    name="g1",
    initial_state={'opening': 0.3}, # 30% open
    parameters={'width': 10, 'max_opening': 1.0, 'discharge_coefficient': 0.6}
)

# 2. Set up the Simulation Harness
simulation_config = {'duration': 600, 'dt': 1.0}
harness = SimulationHarness(config=simulation_config)

# 3. Add components and define topology
harness.add_component(reservoir)
harness.add_component(pipe)
harness.add_component(gate)

harness.add_connection("res1", "pipe1")
harness.add_connection("pipe1", "g1")

# 4. Build and run the simulation
# We will redirect stdout to a file to keep the notebook clean
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    harness.run_simulation()
sys.stdout = original_stdout # Restore stdout

print("Simulation complete. Results are stored in harness.history.")

## Results and Visualization

After running the simulation, the `harness.history` attribute contains a complete record of each component's state at every time step. We can use this data to plot the system's behavior.

We will plot:
- The `water_level` of the reservoir.
- The `outflow` from the pipe and the gate.

In [ ]:
# Process the history data into a more usable format (Pandas DataFrame)
time_steps = [h['time'] for h in harness.history]
reservoir_levels = [h['res1']['water_level'] for h in harness.history]
pipe_outflows = [h['pipe1']['outflow'] for h in harness.history]
gate_outflows = [h['g1']['outflow'] for h in harness.history]

# Create a DataFrame for easy viewing
df = pd.DataFrame({
    'Time (s)': time_steps,
    'Reservoir Level (m)': reservoir_levels,
    'Pipe Outflow (m^3/s)': pipe_outflows,
    'Gate Outflow (m^3/s)': gate_outflows
})

print("Simulation Results:")
print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Plot water level
ax1.plot(df['Time (s)'], df['Reservoir Level (m)'], label='Reservoir Water Level', color='blue')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('Reservoir and Outflow Dynamics')
ax1.grid(True)
ax1.legend()

# Plot outflows
ax2.plot(df['Time (s)'], df['Pipe Outflow (m^3/s)'], label='Pipe Outflow', color='green')
ax2.plot(df['Time (s)'], df['Gate Outflow (m^3/s)'], label='Gate Outflow', color='red', linestyle='--')
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Outflow (m^3/s)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()